# Custom Measurement Tutorial
This tutorial will demonstrate how to encode custom measurements -- such as two-qubit parity measurement into a pyGSTi model -- rather than the standard Z measurement in the computational basis.

In [1]:
import pygsti
from pygsti.modelpacks import smq2Q_XYCNOT as std
import numpy as np

## Parity measurement construction

We start with a standard two-qubit model, and replace the default POVM with one that measures the parity instead. We do this by providing the superkets which described the desired measurement. This is straightforward for the parity measurement in the Pauli product basis, as shown below.

In [2]:
parity_model = std.target_model()

# Here, we specify the superkets for the even/odd effects
# This can be done in any basis, but we use Pauli-product here since
# we know the structure of the parity measurements in this basis
even_dmvec = np.zeros(16)
even_dmvec[0] = 1.0  # II element should be 1
even_dmvec[15] = 1.0 # ZZ element should also be 1 for even

odd_dmvec = np.zeros(16)
odd_dmvec[0] = 1.0  # II element is still 1 for odd...
odd_dmvec[15] = -1.0 # ... but ZZ element should be -1 for odd

parity_povm_dict = {'e': even_dmvec, 'o': odd_dmvec}

parity_povm = pygsti.modelmembers.povms.create_from_dmvecs(parity_povm_dict, "full TP",
    basis='pp', evotype=parity_model.evotype, state_space=parity_model.state_space)

parity_model['Mdefault'] = parity_povm
print(parity_model)

rho0 = FullState with dimension 16
 0.50   0   0 0.50   0   0   0   0   0   0   0   0 0.50   0   0 0.50


Mdefault = TPPOVM with effect vectors:
e: FullPOVMEffect with dimension 16
 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0   0 1.00

o: ComplementPOVMEffect with dimension 16
 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0   0-1.00



Gxpi2:1 = 
FullArbitraryOp with shape (16, 16)
 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0-1.00   0   0   0   0   0   0   0   0   0   0   0   0
   0   0 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 1.00   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0 1.00   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0-1.00   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 1.00   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0 1.00   0   0   0   0   0   0   

We can test this by running some simple circuits and seeing what outcomes we observe.

In [3]:
# Idle circuit should give us even outcome
dict(parity_model.probabilities( pygsti.circuits.Circuit([], line_labels=(0,1))))

{('e',): 1.0000000000000002, ('o',): 0.0}

In [16]:
# Partial flip of one qubit gives an equal superposition of odd and even
dict(parity_model.probabilities( pygsti.circuits.Circuit([('Gxpi2', 0)], line_labels=(0,1))))

{('e',): 1.0000000000000002, ('o',): 0.9999999999999998}

In [5]:
# Full bitflip of one qubit should give us an odd outcome
dict(parity_model.probabilities( pygsti.circuits.Circuit([('Gxpi2', 0), ('Gxpi2', 0)], line_labels=(0,1))))

{('e',): 0.0, ('o',): 1.0}

In [6]:
# Making a Bell pair (using H = Y(pi/2)X(pi), in operation order) should maintain the even outcome
dict(parity_model.probabilities( pygsti.circuits.Circuit([('Gypi2', 0), ('Gxpi2', 0), ('Gxpi2', 0), ('Gcnot', 0, 1)], line_labels=(0,1))))

{('e',): 1.0, ('o',): 0.0}

## Combining measurements

It is also possible to use different measurements on different sets of qubits. For example, we can mix computational basis states with our parity measurement from above.

Since we are going up to 3 qubits for this example, we will swap over to using a `QubitProcessorSpec` and `pygsti.modelconstruction` to build our initial model rather than loading it from a modelpack.

In [14]:
# Get a basic 3-qubit model
pspec = pygsti.processors.QubitProcessorSpec(3, ['Gxpi2', 'Gypi2', 'Gcnot'], geometry='line')
Z_parity_model = pygsti.models.create_explicit_model(pspec)

# Get a 1-qubit Z basis (computational) measurement
computational_povm = pygsti.modelmembers.povms.ComputationalBasisPOVM(1)

# Get a composite POVM that performs Z measurement on qubit 1 and a parity measurement on qubits 2 and 3
Z_parity_povm = pygsti.modelmembers.povms.TensorProductPOVM([computational_povm, parity_povm])

# Override our standard measurement with the composite one
Z_parity_model['Mdefault'] = Z_parity_povm


And we can again test this with some simple measurements. Notice that instead of binary bitstrings, the "e"/"o" outcome labels are used as the second part of the outcome labels.

In [17]:
# Idle circuit should give us 0 on first qubit and even parity on second and third qubits
dict(Z_parity_model.probabilities( pygsti.circuits.Circuit([], line_labels=(0,1,2)) ))

{('0e',): 0.9999999999999997, ('0o',): 0.0, ('1e',): 0.0, ('1o',): 0.0}

In [18]:
# We can flip just the first qubit to see a 1 but still even outcome
dict(Z_parity_model.probabilities( pygsti.circuits.Circuit([('Gxpi2', 0), ('Gxpi2', 0)], line_labels=(0,1,2)) ))

{('0e',): -5.551115123125783e-17,
 ('0o',): -1.6653345369377348e-16,
 ('1e',): 0.9999999999999993,
 ('1o',): -1.6653345369377348e-16}

In [20]:
# Alternatively we can flip the last qubit to get a 0 but odd outcome
dict(Z_parity_model.probabilities( pygsti.circuits.Circuit([('Gxpi2', 2), ('Gxpi2', 2)], line_labels=(0,1,2)) ))

{('0e',): -1.6653345369377348e-16,
 ('0o',): 0.9999999999999996,
 ('1e',): -3.885780586188048e-16,
 ('1o',): -5.551115123125783e-17}

In [21]:
# And we can do partial flip of qubits 0 and 1 to get a uniform spread over all outcome possibilities
dict(Z_parity_model.probabilities( pygsti.circuits.Circuit([('Gxpi2', 0), ('Gxpi2', 1)], line_labels=(0,1,2)) ))

{('0e',): 0.24999999999999992,
 ('0o',): 0.2499999999999998,
 ('1e',): 0.24999999999999986,
 ('1o',): 0.24999999999999975}